In [1]:
import numpy as np
import pandas as pd
import seaborn as sns # libreria utile per matrice di confusione
# import plotly.express as px
from collections import Counter
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, make_scorer, confusion_matrix, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
#import shap
import pickle
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.inspection import PartialDependenceDisplay

In [2]:
plt.rcParams["mathtext.fontset"]
plt.rcParams.update({'font.size': 28, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

pd.set_option('display.max_columns', None)

In [3]:
transaction_frequency_opcode = ['address', 'balance', 'lifetime', 'tx_in', 'tx_out', 'investment_in',
 'payment_out', 'investment_to_contract/tx_in', 'payment_from_contract/tx_out',
 '#addresses_paying_contract', '#addresses_paid_by_contract', 'mean_v1', 'sdev_v1', 'mean_v2', 'sdev_v2', 'paid_rate',
 'paid_one', 'percentage_some_tx_in', 'sdev_tx_in', 'percentage_some_tx_out', 'sdev_tx_out', 'owner_gets_eth_Wo_investing',
 'owner_gets_eth_investing', 'owner_no_eth', 'PUSH', 'DUP', 'JUMPDEST', 'STOP', 'MSTORE', 'JUMPI', 'REVERT', 'CALLVALUE',
 'ISZERO', 'CODECOPY', 'RETURN', 'LOG', 'SHA3', 'MSTORE8', 'SWAP', 'POP', 'ADD', 'MLOAD', 'AND', 'SUB', 'CALLDATALOAD', 'EXP',
 'MUL', 'SLOAD', 'EQ', 'JUMP', 'DIV', 'CALLER', 'CALLDATACOPY', 'SSTORE', 'NOT', 'CALL', 'LT', 'GT', 'OR', 'ADDRESS', 'TIMESTAMP',
 'GASLIMIT', 'GAS', 'ORIGIN', 'BALANCE', 'CALLDATASIZE', 'SAR', 'MSIZE', 'CODESIZE', 'COINBASE', 'CREATE2', 'EXTCODESIZE', 'CALLCODE', 'SHL',
 'BLOCKHASH', 'RETURNDATASIZE', 'SHR', 'GETPC', 'DELEGATECALL', 'MOD', 'ADDMOD', 'NUMBER', 'XOR', 'SLT', 'EXTCODECOPY', 'MULMOD', 'CREATE', 'SELFDESTRUCT',
 'STATICCALL', 'RETURNDATACOPY', 'SGT', 'DIFFICULTY', 'SMOD', 'BYTE', 'SIGNEXTEND', 'CHAINID', 'SELFBALANCE', 'GASPRICE', 'EXTCODEHASH', 'SDIV', 'target']

In [4]:
ds_full_path= (r"C:\Users\lucap\OneDrive\Desktop\Scam on blockchain\Smart Ponzi\DS_deployed_bytecode.csv")
db = pd.read_csv(ds_full_path, header = 0, delimiter = ',')
db = db.fillna(0)
db

,address,balance,lifetime,tx_in,tx_out,investment_in,payment_out,investment_to_contract/tx_in,payment_from_contract/tx_out,#addresses_paying_contract,#addresses_paid_by_contract,mean_v1,sdev_v1,mean_v2,sdev_v2,paid_rate,paid_one,percentage_some_tx_in,sdev_tx_in,percentage_some_tx_out,sdev_tx_out,owner_gets_eth_Wo_investing,owner_gets_eth_investing,owner_no_eth,PUSH,STOP,MSTORE,JUMPDEST,DUP,REVERT,LOG,SHA3,MSTORE8,JUMPI,SWAP,POP,ADD,MLOAD,AND,CALLDATALOAD,SUB,EXP,ISZERO,MUL,SLOAD,EQ,JUMP,DIV,CALLER,CALLDATACOPY,SSTORE,NOT,LT,CALL,GT,CALLVALUE,OR,CODECOPY,ADDRESS,TIMESTAMP,GASLIMIT,GAS,RETURN,CALLDATASIZE,BALANCE,ORIGIN,SAR,MSIZE,CREATE2,EXTCODESIZE,CALLCODE,SHL,BLOCKHASH,DELEGATECALL,NUMBER,MOD,XOR,COINBASE,SLT,ADDMOD,EXTCODECOPY,MULMOD,CREATE,SELFDESTRUCT,STATICCALL,RETURNDATACOPY,SGT,DIFFICULTY,SMOD,BYTE,RETURNDATASIZE,SIGNEXTEND,SHR,GASPRICE,EXTCODEHASH,SDIV,CHAINID,GETPC,SELFBALANCE,CODESIZE,target
0,0x0006157838d5a6b33ab66588a6a693a57c869999,0.000691,117,2,0,1,0,0.5000,0.0,1,0,-1.000000,0.000000,-0.000691,0.000000,0.0000,0.0,0.0171,0.000000,0.0000,0.000000,0,0,1,5.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x001a589dda0d6be37632925eaf1256986b2c6ad0,29.408568,318,11346,4,9685,4,0.8536,1.0,39,1,-242.025000,509.123977,-0.735203,7.530951,0.0004,0.0,0.5252,92.723961,0.0126,0.000000,0,0,1,1023.0,2.0,165.0,90.0,1123.0,0.0,3.0,33.0,0.0,86.0,911.0,468.0,345.0,105.0,95.0,91.0,90.0,77.0,71.0,67.0,58.0,54.0,52.0,45.0,40.0,34.0,18.0,18.0,15.0,15.0,9.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x00674045bb7c17f0aa1cde34780d6c51af548728,151.824186,1366,29,12,2,12,0.0690,1.0,2,7,1.111111,1.448712,-16.869354,765.672046,6.0000,0.0,0.0124,0.455645,0.0088,0.000000,0,0,1,1081.0,10.0,152.0,311.0,656.0,46.0,13.0,55.0,0.0,151.0,494.0,179.0,172.0,95.0,106.0,28.0,101.0,72.0,141.0,28.0,107.0,34.0,101.0,17.0,12.0,1.0,27.0,18.0,36.0,1.0,16.0,28.0,10.0,0.0,5.0,3.0,0.0,1.0,18.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45,0.000000,21,9708,0,0,0,0.0000,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,1.0476,236.751919,0.0000,0.000000,0,0,1,700.0,7.0,115.0,178.0,501.0,12.0,11.0,30.0,0.0,94.0,402.0,107.0,98.0,81.0,110.0,20.0,106.0,63.0,95.0,13.0,48.0,31.0,63.0,13.0,13.0,0.0,18.0,15.0,6.0,2.0,8.0,20.0,3.0,4.0,6.0,0.0,0.0,2.0,14.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0x00a9f7d093c46d95f0318e4a6ffc6ed68f73044c,0.000000,0,78,1,76,1,0.9744,1.0,65,1,-1.136364,0.488857,0.000000,97.843007,0.0132,0.0,1.0000,0.000000,1.0000,0.000000,0,0,1,568.0,8.0,55.0,167.0,286.0,18.0,5.0,4.0,0.0,93.0,249.0,61.0,66.0,74.0,69.0,3.0,68.0,39.0,94.0,8.0,66.0,28.0,58.0,11.0,13.0,0.0,11.0,9.0,7.0,8.0,6.0,30.0,5.0,0.0,3.0,3.0,0.0,6.0,18.0,1.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7441,0xfd784da5c740c617aafb80399fa81b86e1da99a5,0.000000,461,9964,0,0,0,0.0000,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0000,0

Daset Preparation

In [5]:
df_transaction_weighted_opcode = db[transaction_frequency_opcode].copy()

y = df_transaction_weighted_opcode.loc[:, ['target']].values
y = pd.DataFrame(y,columns=['target'])

dataset_account = db.iloc[:, 0:24]
dataset_opcode = db.iloc[:, 24:100]

dataset_opcode['all_opcode'] = dataset_opcode[list(dataset_opcode.columns)].sum(axis=1) #generate all_opcode
dataset_opcode = dataset_opcode[:].div(dataset_opcode['all_opcode'], axis=0)
dataset_opcode = dataset_opcode.drop(['all_opcode'], axis = 1)

df_transaction_weighted_opcode = pd.concat([dataset_account, dataset_opcode, y], axis=1)
df_transaction_weighted_opcode = df_transaction_weighted_opcode.fillna(0)

df_transaction_weighted_opcode

,address,balance,lifetime,tx_in,tx_out,investment_in,payment_out,investment_to_contract/tx_in,payment_from_contract/tx_out,#addresses_paying_contract,#addresses_paid_by_contract,mean_v1,sdev_v1,mean_v2,sdev_v2,paid_rate,paid_one,percentage_some_tx_in,sdev_tx_in,percentage_some_tx_out,sdev_tx_out,owner_gets_eth_Wo_investing,owner_gets_eth_investing,owner_no_eth,PUSH,STOP,MSTORE,JUMPDEST,DUP,REVERT,LOG,SHA3,MSTORE8,JUMPI,SWAP,POP,ADD,MLOAD,AND,CALLDATALOAD,SUB,EXP,ISZERO,MUL,SLOAD,EQ,JUMP,DIV,CALLER,CALLDATACOPY,SSTORE,NOT,LT,CALL,GT,CALLVALUE,OR,CODECOPY,ADDRESS,TIMESTAMP,GASLIMIT,GAS,RETURN,CALLDATASIZE,BALANCE,ORIGIN,SAR,MSIZE,CREATE2,EXTCODESIZE,CALLCODE,SHL,BLOCKHASH,DELEGATECALL,NUMBER,MOD,XOR,COINBASE,SLT,ADDMOD,EXTCODECOPY,MULMOD,CREATE,SELFDESTRUCT,STATICCALL,RETURNDATACOPY,SGT,DIFFICULTY,SMOD,BYTE,RETURNDATASIZE,SIGNEXTEND,SHR,GASPRICE,EXTCODEHASH,SDIV,CHAINID,GETPC,SELFBALANCE,CODESIZE,target
0,0x0006157838d5a6b33ab66588a6a693a57c869999,0.000691,117,2,0,1,0,0.5000,0.0,1,0,-1.000000,0.000000,-0.000691,0.000000,0.0000,0.0,0.0171,0.000000,0.0000,0.000000,0,0,1,0.333333,0.133333,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0x001a589dda0d6be37632925eaf1256986b2c6ad0,29.408568,318,11346,4,9685,4,0.8536,1.0,39,1,-242.025000,509.123977,-0.735203,7.530951,0.0004,0.0,0.5252,92.723961,0.0126,0.000000,0,0,1,0.195602,0.000382,0.031549,0.017208,0.214723,0.000000,0.000574,0.006310,0.000000,0.016444,0.174187,0.089484,0.065966,0.020076,0.018164,0.017400,0.017208,0.014723,0.013576,0.012811,0.011090,0.010325,0.009943,0.008604,0.007648,0.006501,0.003442,0.003442,0.002868,0.002868,0.001721,0.000765,0.000765,0.000765,0.000574,0.000382,0.000382,0.000382,0.000382,0.000191,0.000191,0.000191,0.000191,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0x00674045bb7c17f0aa1cde34780d6c51af548728,151.824186,1366,29,12,2,12,0.0690,1.0,2,7,1.111111,1.448712,-16.869354,765.672046,6.0000,0.0,0.0124,0.455645,0.0088,0.000000,0,0,1,0.249366,0.002307,0.035063,0.071742,0.151326,0.010611,0.002999,0.012687,0.000000,0.034833,0.113956,0.041292,0.039677,0.021915,0.024452,0.006459,0.023299,0.016609,0.032526,0.006459,0.024683,0.007843,0.023299,0.003922,0.002768,0.000231,0.006228,0.004152,0.008304,0.000231,0.003691,0.006459,0.002307,0.000000,0.001153,0.000692,0.000000,0.000231,0.004152,0.000231,0.000000,0.000000,0.000000,0.001845,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0x006bea43baa3f7a6f765f14f10a1a1b08334ef45,0.000000,21,9708,0,0,0,0.0000,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,1.0476,236.751919,0.0000,0.000000,0,0,1,0.232172,0.002322,0.038143,0.059038,0.166169,0.003980,0.003648,0.009950,0.000000,0.031177,0.133333,0.035489,0.032504,0.026866,0.036484,0.006633,0.035158,0.020896,0.031509,0.004312,0.015920,0.010282,0.020896,0.004312,0.004312,0.000000,0.005970,0.004975,0.001990,0.000663,0.002653,0.006633,0.000995,0.001327,0.001990,0.000000,0.000000,0.000663,0.004643,0.000332,0.000000,0.000000,0.000000,0.000000,0.000663,0.000332,0.000332,0.000332,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [6]:
col_name = transaction_frequency_opcode[1:-1]

In [7]:
X = df_transaction_weighted_opcode.iloc[:, 1:-1].values # rimuovo la prima colonna che contiene gli indirizzi
y = df_transaction_weighted_opcode.loc[:, ['target']].values

X = pd.DataFrame(X)
X.columns = col_name

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 42, stratify=y) 

In [9]:
params =[

        {
            'classifier': [lgb.LGBMClassifier(boosting_type='gbdt', n_jobs=4, importance_type='split',random_state=42)],
            'classifier__learning_rate' : [0.1], #[0.1,0.01]
            'classifier__n_estimators' : [100], #[80, 100, 120]
            'classifier__max_depth' : [15], #[10,15,20],
            'classifier__colsample_bytree' : [0.5], #[0.5,0.8,1],
            'classifier__reg_alpha' : [0], #[0, 0.1, 0.2],
            'classifier__reg_lambda' : [1], #[1,10,15],
        }
]

In [10]:
data = {}
pipeline = Pipeline(steps=[('classifier',RandomForestClassifier(random_state=42))])

grid_search = GridSearchCV(pipeline, params, cv=5,
                               scoring='roc_auc',
                               return_train_score=True,n_jobs=4,verbose=2)

Features Selection

In [11]:
cross_validations = {}

df_cross_validations = pd.DataFrame()

df_X_train_full = X_train
df_X_test_full = X_test
# model = grid_search.best_estimator_
model = grid_search
auc_scores = {'n_features':[],'auc_score':[]}
for i in range(len(df_X_train_full.columns),10,-1):
    rfe = RFE(model,n_features_to_select=i)
    rfe = rfe.fit(df_X_train_full, y_train.ravel())

# summarize the selection of the attributes
#     print(rfe.support_)
#     print(rfe.ranking_)
    print(set(rfe.feature_names_in_).difference(rfe.get_feature_names_out()))
    X_train_transformed = rfe.transform(df_X_train_full)
    X_test_transformed = rfe.transform(df_X_test_full)
    
    df_X_train_full = pd.DataFrame(X_train_transformed,columns=rfe.get_feature_names_out())
    df_X_test_full = pd.DataFrame(X_test_transformed,columns=rfe.get_feature_names_out())
    
    pipeline = Pipeline(steps=[('classifier',RandomForestClassifier(random_state=42))])

    params =[

             {
          'classifier': [lgb.LGBMClassifier(boosting_type='gbdt', n_jobs=4, importance_type='split',random_state=42)],
          'classifier__learning_rate' : [0.1,0.01],
          'classifier__n_estimators' : [100, 120, 140],
          'classifier__max_depth' : [10,15,20],
          'classifier__colsample_bytree' : [0.5,0.8,1],
          'classifier__reg_alpha' : [0, 0.1, 0.2],
          'classifier__reg_lambda' : [1,10,15]
        }



     ]
    grid_search = GridSearchCV(pipeline, params, cv=5,
                           scoring='roc_auc',
                           return_train_score=True,n_jobs=4)
    grid_search.fit(df_X_train_full, y_train.ravel())
    cross_validations[i] = grid_search 
    _tmp_cv = pd.DataFrame(grid_search.cv_results_)
    _tmp_cv['n_features'] = i
    _tmp_cv['features'] = ','.join([column for column in rfe.get_feature_names_out() ])
    
    df_cross_validations = pd.concat([df_cross_validations,_tmp_cv])
    
    model = grid_search.best_estimator_.named_steps['classifier']
    model.fit(df_X_train_full,y_train.ravel())
    y_pred_proba = model.predict_proba(df_X_test_full)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba)    
    auc_score = roc_auc_score(y_test,y_pred_proba)
    print(auc_score)
    auc_scores['n_features'].append(i)
    auc_scores['auc_score'].append(auc_score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
set()
0.9757883402553454
{'GASPRICE'}
0.9757575757575758
{'ISZERO'}
0.9789417012767266
{'CREATE2'}
0.9780264574680818
{'SIGNEXTEND'}
0.979103214890017
{'DIFFICULTY'}
0.976465159206276
{'ADDMOD'}
0.9765959083217967
{'NUMBER'}
0.9749192431933548
{'owner_gets_eth_investing'}
0.979318566374404
{'CALLDATASIZE'}
0.9741885863713275
{'TIMESTAMP'}
0.9740424550069219
{'payment_from_contract/tx_out'}
0.9765959083217965
{'SELFBALANCE'}
0.9759652361175204
{'SAR'}
0.9765343793262575
{'STATICCALL'}
0.9780956775880634
{'BYTE'}
0.9788417166589756
{'EXTCODESIZE'}
0.9771958160283034
{'EXTCODEHASH'}
0.9756883556375943
{'owner_no_eth'}
0.9753807106598986
{'#addresses_paying_contract'}
0.9750576834333179
{'CALLCODE'}
0.9726042147361944
{'owner_gets_eth_Wo_investing'}
0.9753576372865713
{'SELFDESTRUCT'}
0.9739963082602677
{'EXTCODECOPY'}
0.9781110598369481
{'RETURNDATACOPY'}
0.97599600061529
{'DELEGATECALL'}
0.9748192585756037
{'tx_out'}
0.9743808644

In [12]:
df_cross_validations.sort_values('mean_test_score',ascending=False).head(1)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__colsample_bytree,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__reg_alpha,param_classifier__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_features,features
18,4.208447,0.15109,0.032729,0.007693,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980767,0.988148,0.971362,0.989821,0.965457,0.979111,0.00944,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,65,"balance,lifetime,tx_in,payment_out,investment_..."


In [29]:
pd.set_option('display.max_columns', None)

Ho selezionato il 'terzo' migliore in quanto le performance erano allineate ma la terza scelta utilizzava meno della metà delle features

In [13]:
df_cross_validations.sort_values('mean_test_score',ascending=False).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__colsample_bytree,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__reg_alpha,param_classifier__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_features,features
18,4.208447,0.151090,0.032729,0.007693,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980767,0.988148,0.971362,0.989821,0.965457,0.979111,0.009440,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
9,3.741218,0.172507,0.028223,0.004726,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,120,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980905,0.987969,0.970741,0.989707,0.966172,0.979099,0.009289,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
72,2.514835,0.070963,0.014549,0.004267,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.981620,0.988513,0.967578,0.989737,0.968013,0.979092,0.009631,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,86,"balance,lifetime,tx_in,tx_out,investment_in,pa..."
63,2.137100,0.071331,0.015804,0.004182,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,20,120,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.981386,0.988650,0.967458,0.989767,0.968067,0.979066,0.009669,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,86,"balance,lifetime,tx_in,tx_out,investment_in,pa..."
0,4.725851,0.709019,0.029670,0.008459,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,100,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980623,0.988011,0.970520,0.989683,0.966232,0.979014,0.009305,3,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
234,2.690789,0.102073,0.023868,0.003225,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.979529,0.988566,0.971691,0.988053,0.967116,0.978991,0.008584,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,49,"lifetime,tx_in,investment_to_contract/tx_in,me..."
207,3.719676,0.265128,0.022503,0.001240,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,15,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.979529,0.988566,0.971691,0.987443,0.967417,0.978929,0.008374,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,49,"lifetime,tx_in,investment_to_contract/tx_in,me..."
234,2.619058,0.086758,0.023560,0.009992,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980334,0.988871,0.970296,0.986434,0.968668,0.978921,0.008209,1,1.0,1.0,1.0,1.0,1.0,1.0,9.169462e-08,43,"tx_in,investment_to_contract/tx_in,mean_v1,sde..."
207,2.553553,0.126041,0.020159,0.005746,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,15,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980334,0.988542,0.970296,0.985669,0.969720,0.978912,0.007735,2,1.0,1.0,1.0,1.0,1.0,1.0,9.169462e-08,43,"tx_in,investment_to_contract/tx_in,mean_v1,sde..."
45,4.723016,0.208299,0.033103,0.004711,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,15,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980881,0.984857,0.970666,0.989695,0.968235,0.978867,0.008216,1,1.0,1.0,1.0,1.0,1.0,1.0,4.965068e-17,91,"balance,lifetime,tx_in,tx_out,investment_in,pa..."


In [14]:
tab_res_ = df_cross_validations.sort_values('mean_test_score',ascending=False).head(8)

In [15]:
tab_res_

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__colsample_bytree,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__reg_alpha,param_classifier__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_features,features
18,4.208447,0.151090,0.032729,0.007693,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980767,0.988148,0.971362,0.989821,0.965457,0.979111,0.009440,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
9,3.741218,0.172507,0.028223,0.004726,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,120,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980905,0.987969,0.970741,0.989707,0.966172,0.979099,0.009289,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
72,2.514835,0.070963,0.014549,0.004267,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.981620,0.988513,0.967578,0.989737,0.968013,0.979092,0.009631,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,86,"balance,lifetime,tx_in,tx_out,investment_in,pa..."
63,2.137100,0.071331,0.015804,0.004182,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,20,120,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.981386,0.988650,0.967458,0.989767,0.968067,0.979066,0.009669,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,86,"balance,lifetime,tx_in,tx_out,investment_in,pa..."
0,4.725851,0.709019,0.029670,0.008459,"LGBMClassifier(colsample_bytree=0.5, max_depth...",0.5,0.1,10,100,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980623,0.988011,0.970520,0.989683,0.966232,0.979014,0.009305,3,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,65,"balance,lifetime,tx_in,payment_out,investment_..."
234,2.690789,0.102073,0.023868,0.003225,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.979529,0.988566,0.971691,0.988053,0.967116,0.978991,0.008584,1,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,49,"lifetime,tx_in,investment_to_contract/tx_in,me..."
207,3.719676,0.265128,0.022503,0.001240,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,15,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.979529,0.988566,0.971691,0.987443,0.967417,0.978929,0.008374,2,1.0,1.0,1.0,1.0,1.0,1.0,0.000000e+00,49,"lifetime,tx_in,investment_to_contract/tx_in,me..."
234,2.619058,0.086758,0.023560,0.009992,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980334,0.988871,0.970296,0.986434,0.968668,0.978921,0.008209,1,1.0,1.0,1.0,1.0,1.0,1.0,9.169462e-08,43,"tx_in,investment_to_contract/tx_in,mean_v1,sde..."


In [16]:
tab_res_ = tab_res_.tail(1)
tab_res_

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__colsample_bytree,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__reg_alpha,param_classifier__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_features,features
234,2.619058,0.086758,0.02356,0.009992,"LGBMClassifier(colsample_bytree=0.8, max_depth...",0.8,0.1,20,140,0,1,{'classifier': LGBMClassifier(colsample_bytree...,0.980334,0.988871,0.970296,0.986434,0.968668,0.978921,0.008209,1,1.0,1.0,1.0,1.0,1.0,1.0,9.169462e-08,43,"tx_in,investment_to_contract/tx_in,mean_v1,sde..."


In [28]:
best_features_set = tab_res_['features']

In [46]:
best_features_set = tab_res_['features'].values[0]

In [47]:
best_features_set = best_features_set.split(',')

In [48]:
best_features_set

['tx_in',
 'investment_to_contract/tx_in',
 'mean_v1',
 'sdev_v1',
 'paid_one',
 'PUSH',
 'DUP',
 'JUMPDEST',
 'STOP',
 'MSTORE',
 'JUMPI',
 'REVERT',
 'CALLVALUE',
 'CODECOPY',
 'RETURN',
 'LOG',
 'SHA3',
 'MSTORE8',
 'SWAP',
 'POP',
 'ADD',
 'MLOAD',
 'AND',
 'SUB',
 'CALLDATALOAD',
 'EXP',
 'MUL',
 'SLOAD',
 'EQ',
 'DIV',
 'CALLER',
 'CALLDATACOPY',
 'SSTORE',
 'NOT',
 'CALL',
 'LT',
 'ADDRESS',
 'GASLIMIT',
 'GAS',
 'ORIGIN',
 'BALANCE',
 'MULMOD',
 'SGT']

In [49]:
best_param_set = df_cross_validations['param_classifier'].values[2]

In [49]:
best_param_set = tab_res_['params']

In [51]:
tab_res_['params'].values[0]

{'classifier': LGBMClassifier(colsample_bytree=0.8, max_depth=20, n_estimators=140, n_jobs=4,
                random_state=42, reg_alpha=0, reg_lambda=1),
 'classifier__colsample_bytree': 0.8,
 'classifier__learning_rate': 0.1,
 'classifier__max_depth': 20,
 'classifier__n_estimators': 140,
 'classifier__reg_alpha': 0,
 'classifier__reg_lambda': 1}

In [53]:
df_cross_validations[:].values[7]

array([2.314264345169067, 0.06456921155239889, 0.0232637882232666,
       0.002594644002292795,
       LGBMClassifier(colsample_bytree=0.8, max_depth=15, n_estimators=140, n_jobs=4,
                      random_state=42, reg_alpha=0.1, reg_lambda=1)                  ,
       0.5, 0.1, 10, 100, 0.2, 10,
       {'classifier': LGBMClassifier(colsample_bytree=0.8, max_depth=15, n_estimators=140, n_jobs=4,
                      random_state=42, reg_alpha=0.1, reg_lambda=1), 'classifier__colsample_bytree': 0.5, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 10, 'classifier__n_estimators': 100, 'classifier__reg_alpha': 0.2, 'classifier__reg_lambda': 10},
       0.9781954299928499, 0.9877538829151733, 0.9658870967741936,
       0.9876762246117085, 0.9609944432416829, 0.9761014155071216,
       0.011015389392811775, 196, 0.9993616152368643, 0.9995003876302868,
       0.9996996333429461, 0.9996030066627467, 0.9996758314996812,
       0.999568094874505, 0.00012437563565448055, 99,
   

In [54]:
best_param_set

234    {'classifier': LGBMClassifier(colsample_bytree...
Name: params, dtype: object

In [55]:
df_X_train_full = pd.DataFrame(X_train,columns=df_transaction_weighted_opcode.columns[1:-1])
df_X_test_full = pd.DataFrame(X_test,columns=df_transaction_weighted_opcode.columns[1:-1])

df_X_train_full = df_X_train_full[best_features_set].copy()
df_X_test_full = df_X_test_full[best_features_set].copy()

model.fit(df_X_train_full,y_train.ravel())
y_pred_proba_best = model.predict_proba(df_X_test_full)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test,y_pred_proba_best)    
auc_score = roc_auc_score(y_test,y_pred_proba_best)


y_pred_best = model.predict(df_X_test_full)
best_precision_test= precision_score(y_test, y_pred_best) # TP / (FP + TP)
best_recall_test = recall_score(y_test, y_pred_best)	     # TP / (FN + TP)
best_accuracy_test = accuracy_score(y_test, y_pred_best)	 # (TP + TN)/ (TP + FN + TN + FP)
best_f1_test = f1_score(y_test, y_pred_best)

#print('AUC: {:.3f}'.format(auc_score))
print('Precision: {:.3f}'.format(best_precision_test))
print('Recall: {:.3f}'.format(best_recall_test))
print('Accuracy: {:.3f}'.format(best_accuracy_test))
print('F1: {:.3f}'.format(best_f1_test))

Precision: 0.929
Recall: 0.788
Accuracy: 0.968
F1: 0.852
